# Assignment4 RAG (GPT MINI)

In [24]:
from langchain.chat_models.ollama import ChatOllama
from langchain.chat_models.openai import ChatOpenAI
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

LLM_model, models = ["openai", "gpt-4o-mini-2024-07-18"]
# LLM_model, models = ["ollama", "gemma2:latest"]

file_name = "document.txt"

llm = (
    ChatOllama(temperature=0.1, model=models)
    if LLM_model == "ollama"
    else ChatOpenAI(temperature=0.1, model=models)
)

loader = UnstructuredFileLoader(f"../files/{file_name}")
cache_dir = LocalFileStore(f"../.cache/embeddings/{LLM_model}/{models}/{file_name}")

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    separators=["\n\n", ".", "?", "!"],
    chunk_size=1000,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)
embeddings = (
    OllamaEmbeddings(model=models) if LLM_model == "ollama" else OpenAIEmbeddings()
)

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
    memory_key="history",
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            당신은 문서를 읽고 대답해주는 AI입니다. 주어지는 문서를 통해 답변을 주세요. 
            만약 정보가 문서에 없다면, '모르겠습니다'라고 답변해주세요.
            한국어로 답변해주세요.
            
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": RunnableLambda(load_memory),
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question).content
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print(result)

In [25]:
invoke_chain("Is Aaronson guilty?")

문서에 따르면, Winston은 Aaronson이 범죄로 기소된 것에 대해 유죄라고 믿고 있으며, 그가 범죄를 저지른 증거가 없다는 사실을 기억하지 못한다고 언급하고 있습니다. 따라서 문서의 내용에 따르면 Aaronson은 유죄로 간주되고 있습니다.


In [26]:
invoke_chain("What message did he write in the table?")

Winston은 테이블에 "FREEDOM IS SLAVERY"와 그 아래에 "TWO AND TWO MAKE FIVE"라는 메시지를 썼습니다.


In [27]:
invoke_chain("Who is Julia?")

Julia는 Winston의 사랑하는 사람으로, 그들의 관계는 문서에서 중요한 주제 중 하나입니다. 그녀는 Winston과 함께 자유롭고 반항적인 감정을 나누었지만, 후에 그들의 관계는 고통과 배신으로 이어지게 됩니다.


In [28]:
invoke_chain("What was the first question I asked?")

invoke_chain("What was the second question I asked?")

invoke_chain("What was the third question I asked?")

당신이 처음으로 한 질문은 "Aaronson guilty?"였습니다.
당신이 두 번째로 한 질문은 "What message did he write in the table?"였습니다.
당신이 세 번째로 한 질문은 "Who is Julia?"였습니다.


In [29]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='문서에 따르면, Winston은 Aaronson이 범죄로 기소된 것에 대해 유죄라고 믿고 있으며, 그가 범죄를 저지른 증거가 없다는 사실을 기억하지 못한다고 언급하고 있습니다. 따라서 문서의 내용에 따르면 Aaronson은 유죄로 간주되고 있습니다.'), HumanMessage(content='What message did he write in the table?'), AIMessage(content='Winston은 테이블에 "FREEDOM IS SLAVERY"와 그 아래에 "TWO AND TWO MAKE FIVE"라는 메시지를 썼습니다.'), HumanMessage(content='Who is Julia?'), AIMessage(content='Julia는 Winston의 사랑하는 사람으로, 그들의 관계는 문서에서 중요한 주제 중 하나입니다. 그녀는 Winston과 함께 자유롭고 반항적인 감정을 나누었지만, 후에 그들의 관계는 고통과 배신으로 이어지게 됩니다.'), HumanMessage(content='What was the first question I asked?'), AIMessage(content='당신이 처음으로 한 질문은 "Aaronson guilty?"였습니다.'), HumanMessage(content='What was the second question I asked?'), AIMessage(content='당신이 두 번째로 한 질문은 "What message did he write in the table?"였습니다.'), HumanMessage(content='What was the third question I asked?'), AIMessage(content='당신이 세 번째로 한 질문은 "Who is J